In [1]:
import pandas as pd

### Clean Population Estimates

In [19]:
# Read in Census Population 2019 estimates
pop = pd.read_csv("data/county_pop.csv", encoding='latin-1')
pop = pop[["STATE", "COUNTY", "STNAME", "CTYNAME", "POPESTIMATE2019"]]

In [20]:
# Define states im our model
ne_states = ['New York', 'Connecticut', 'Massachusetts', 'New Jersey', 'New Hampshire',
            'Rhode Island', 'Maine', 'Vermont']

In [21]:
# Keep only NE states
pop = pop[pop["STNAME"].isin(ne_states)]

In [22]:
# Define functions to generate fips code
def fix_state_code(code):
    if len(str(code)) != 2:
        new_code = "0" + str(code)
        return new_code
    else:
        return str(code)

In [23]:
def fix_county_code(code):
    if len(str(code)) == 1:
        new_code = "00" + str(code)
        return new_code
    elif len(str(code)) == 2:
        new_code = "0" + str(code)
        return new_code
    else:
        return str(code)

In [24]:
pop['STATE_CD'] = pop.apply(lambda row : fix_state_code(row['STATE']), axis = 1) 
pop['COUNTY_CD'] = pop.apply(lambda row : fix_county_code(row['COUNTY']), axis = 1)
pop['FIPS'] = pop['STATE_CD'] + pop['COUNTY_CD']
pop = pop[['STNAME', 'CTYNAME', 'POPESTIMATE2019', 'FIPS']].reset_index(drop=True)

In [25]:
# Remove full state
pop = pop[pop["STNAME"] != pop["CTYNAME"]]

In [26]:
pop.head()

,STNAME,CTYNAME,POPESTIMATE2019,FIPS
1,Connecticut,Fairfield County,943332,09001
2,Connecticut,Hartford County,891720,09003
3,Connecticut,Litchfield County,180333,09005
4,Connecticut,Middlesex County,162436,09007
5,Connecticut,New Haven County,854757,09009


In [28]:
pop.shape

(150, 4)

In [27]:
# Save as cleaned csv
#pop.to_csv("data/cleaned_county_pop.csv")

### Clean Land Area

In [15]:
land = pd.read_excel("data/county_land_data.xls", dtype={'STCOU': object})

In [16]:
land.head()

,Areaname,STCOU,LND010190F,LND010190D,LND010190N1,LND010190N2,LND010200F,LND010200D,LND010200N1,LND010200N2,...,LND110210N1,LND110210N2,LND210190F,LND210190D,LND210190N1,LND210190N2,LND210200F,LND210200D,LND210200N1,LND210200N2
0,UNITED STATES,00000,0,3787425.08,0,0,0,3794083.06,0,0,...,0,0,0,251083.35,0,0,0,256644.62,0,0
1,ALABAMA,01000,0,52422.94,0,0,0,52419.02,0,0,...,0,0,0,1672.71,0,0,0,1675.01,0,0
2,"Autauga, AL",01001,0,604.49,0,0,0,604.45,0,0,...,0,0,0,8.48,0,0,0,8.48,0,0
3,"Baldwin, AL",01003,0,2027.08,0,0,0,2026.93,0,0,...,0,0,0,430.55,0,0,0,430.58,0,0
4,"Barbour, AL",01005,0,904.59,0,0,0,904.52,0,0,...,0,0,0,19.59,0,0,0,19.61,0,0


In [17]:
# Only keep columns we want: land area in square miles (2010)
land = land[["Areaname", "STCOU", "LND110210D"]]

In [18]:
land.head()

,Areaname,STCOU,LND110210D
0,UNITED STATES,00000,3531905.43
1,ALABAMA,01000,50645.33
2,"Autauga, AL",01001,594.44
3,"Baldwin, AL",01003,1589.78
4,"Barbour, AL",01005,884.88


In [29]:
# Merge with population counts
merged = pop.merge(land, how='inner', left_on="FIPS", right_on = "STCOU")

In [30]:
merged.head()

,STNAME,CTYNAME,POPESTIMATE2019,FIPS,Areaname,STCOU,LND110210D
0,Connecticut,Fairfield County,943332,09001,"Fairfield, CT",09001,624.89
1,Connecticut,Hartford County,891720,09003,"Hartford, CT",09003,735.10
2,Connecticut,Litchfield County,180333,09005,"Litchfield, CT",09005,920.56
3,Connecticut,Middlesex County,162436,09007,"Middlesex, CT",09007,369.30
4,Connecticut,New Haven County,854757,09009,"New Haven, CT",09009,604.51


In [31]:
merged.shape

(150, 7)

In [32]:
# Drop redundant columns and rename
merged = merged[["FIPS", "STNAME", "CTYNAME", "POPESTIMATE2019", "LND110210D"]]

In [34]:
merged.columns = ["fips", "state", "county", "pop", "land_area"]

In [35]:
merged.head()

,fips,state,county,pop,land_area
0,09001,Connecticut,Fairfield County,943332,624.89
1,09003,Connecticut,Hartford County,891720,735.10
2,09005,Connecticut,Litchfield County,180333,920.56
3,09007,Connecticut,Middlesex County,162436,369.30
4,09009,Connecticut,New Haven County,854757,604.51


In [36]:
# Calculate population density
merged["pop_density"] = merged["pop"]/merged["land_area"]

In [37]:
merged.head()

,fips,state,county,pop,land_area,pop_density
0,09001,Connecticut,Fairfield County,943332,624.89,1509.596889
1,09003,Connecticut,Hartford County,891720,735.10,1213.059448
2,09005,Connecticut,Litchfield County,180333,920.56,195.894890
3,09007,Connecticut,Middlesex County,162436,369.30,439.848362
4,09009,Connecticut,New Haven County,854757,604.51,1413.966684


In [39]:
# Write to csv
merged.to_csv("data/cleaned_county_pop_density.csv", index=False)